## House Plant Feature Engineering

This notebook will cover the feature engineering process (step 2) on the houseplant database generated for two goals: 
- Perform dimensionality reduction (step 3) as a way to visualise similar plants (ultimate goal to make some 2D scatter like graphs doing this). 
- Build a content based recommender system with this dataset (Step 4).


**Summary of This Notebook:**

- TODO

#### Setup

In [1]:
import pandas as pd
import sqlite3

DATABASE_LOC = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\PyForFun\House_Plant_Recommender\Database\house_plants.db"

In [2]:
conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
plant_df = pd.read_sql_query("SELECT * FROM plant_raw_data", conn)
c.close()
plant_df.head()

,Plant_Name,Common_Names,Plant_Type,Family,Zones,Native_Range,Heights,Spreads,Bloom_Times,Bloom_Description,Sunlight,Watering,Maintenance,Flowers,Leafs,Fruits
0,Aechmea,"urn plant,silver vase plant",Epiphyte,Bromeliaceae,10 to 11,Brazil,1.00 to 3.00 feet,1.00 to 2.00 feet,Seasonal bloomer,Violet to red with pink bracts,Part shade,Medium,Medium,Showy,Evergreen,None
1,Ardisia crenata,"hen's eyes,coralberry,spiceberry,scratchthroat...",Broadleaf evergreen,Primulaceae,8 to 10,Japan to Northern India,4.00 to 5.00 feet,4.00 to 5.00 feet,May to June,Pinkish-white,Part shade to full shade,Medium,Medium,Showy,Evergreen,Showy
2,Euphorbia milii,"christplant,Christ plant,Christ thorn,crown of...",Broadleaf evergreen,Euphorbiaceae,9 to 11,Madagascar,3.00 to 6.00 feet,1.50 to 3.00 feet,Seasonal bloomer,Green subtended by red or yellow bracts,Full sun,Dry to medium,Medium,Showy,Evergreen,None
3,Ficus elastica,"Indian rubberplant,India rubber plant,rubber p...",Broadleaf evergreen,Moraceae,10 to 12,Southeastern Asia,50.00 to 100.00 feet,50.00 to 100.00 feet,Rarely flowers indoors,None,Part shade,Medium,Low,Insignificant,Evergreen,None
4,Woodsia obtusa,"blunt-lobed woodsia,common woodsia,large woodsia",Fern,Woodsiaceae,4 to 8,North America,1.00 to 1.50 feet,2.00 to 2.50 feet,Non-flowering,Non-flowering,Part shade to full shade,Medium,Medium,None,None,None


In [3]:
len(plant_df)

147

In [4]:
plant_df.columns

Index(['Plant_Name', 'Common_Names', 'Plant_Type', 'Family', 'Zones',
       'Native_Range', 'Heights', 'Spreads', 'Bloom_Times',
       'Bloom_Description', 'Sunlight', 'Watering', 'Maintenance', 'Flowers',
       'Leafs', 'Fruits'],
      dtype='object')

#### DF Clean up 


In [5]:
# remove the unwanted columns 
drop_list = ["Common_Names", "Family", "Native_Range", "Flowers", "Bloom_Description"]
plant_df = plant_df.drop(drop_list, axis=1)
plant_df.head()

,Plant_Name,Plant_Type,Zones,Heights,Spreads,Bloom_Times,Sunlight,Watering,Maintenance,Leafs,Fruits
0,Aechmea,Epiphyte,10 to 11,1.00 to 3.00 feet,1.00 to 2.00 feet,Seasonal bloomer,Part shade,Medium,Medium,Evergreen,None
1,Ardisia crenata,Broadleaf evergreen,8 to 10,4.00 to 5.00 feet,4.00 to 5.00 feet,May to June,Part shade to full shade,Medium,Medium,Evergreen,Showy
2,Euphorbia milii,Broadleaf evergreen,9 to 11,3.00 to 6.00 feet,1.50 to 3.00 feet,Seasonal bloomer,Full sun,Dry to medium,Medium,Evergreen,None
3,Ficus elastica,Broadleaf evergreen,10 to 12,50.00 to 100.00 feet,50.00 to 100.00 feet,Rarely flowers indoors,Part shade,Medium,Low,Evergreen,None
4,Woodsia obtusa,Fern,4 to 8,1.00 to 1.50 feet,2.00 to 2.50 feet,Non-flowering,Part shade to full shade,Medium,Medium,None,None


In [6]:
# Two plants ("Asplenium antiquum", "Basella alba") had missing info, so will manually add that now as it is available in paragraph form from the database. 
plant_row = (plant_df["Plant_Name"].loc[lambda x: x=="Asplenium antiquum"].index)
display(plant_df.iloc[plant_row])

plant_row = (plant_df["Plant_Name"].loc[lambda x: x=="Basella alba"].index)
display(plant_df.iloc[plant_row])


,Plant_Name,Plant_Type,Zones,Heights,Spreads,Bloom_Times,Sunlight,Watering,Maintenance,Leafs,Fruits
23,Asplenium antiquum,Fern,10 to 11,1.50 to 2.00 feet,3.00 to 4.00 feet,Non-flowering,None,None,None,None,None


,Plant_Name,Plant_Type,Zones,Heights,Spreads,Bloom_Times,Sunlight,Watering,Maintenance,Leafs,Fruits
75,Basella alba,Vine,None,2.00 to 6.00 feet,2.00 to 3.00 feet,July to frost,Full sun,Medium,Medium,None,None


In [7]:
plant_df.at[23, "Sunlight"] = "Part shade"
plant_df.at[23, "Watering"] = "Medium"
plant_df.at[23, "Maintenance"] = "Low"
plant_df.at[23, "Leafs"] = "None" # Can be kept as none as this indicates non-colorful leaves which is correct. 
plant_df.at[23, "Fruits"] = "None" # As above, does not produce fruit. 

plant_df.at[75, "Zones"] = "6 to 10" 

In [8]:
# confirm substitution worked. 
plant_df.iloc[plant_row]

,Plant_Name,Plant_Type,Zones,Heights,Spreads,Bloom_Times,Sunlight,Watering,Maintenance,Leafs,Fruits
75,Basella alba,Vine,6 to 10,2.00 to 6.00 feet,2.00 to 3.00 feet,July to frost,Full sun,Medium,Medium,None,None


#### Engineering Time - Column: "Plant_Type"

In [9]:
plant_df["Plant_Type"].value_counts()

Herbaceous perennial    49
Broadleaf evergreen     33
Bulb                    16
Vine                    13
Fern                    10
Deciduous shrub          7
Palm or Cycad            4
Orchid                   4
Epiphyte                 3
Needled evergreen        2
Fruit                    2
Tree                     2
Annual                   1
Rush or Sedge            1
Name: Plant_Type, dtype: int64

In [10]:
# Will keep the top 5 (up to Fern) and move all others into the group "Other". 
keep_me = ["Herbaceous perennial", "Broadleaf evergreen", "Bulb", "Vine", "Fern"]

plant_types = list(plant_df["Plant_Type"])

new_plant_types = []
for plant in plant_types:
    if plant not in keep_me:
        new_plant_types.append("Other")
    else: 
        new_plant_types.append(plant)

plant_df["Plant_Type"] = new_plant_types

In [11]:
# Confirm worked
plant_df["Plant_Type"].value_counts()

Herbaceous perennial    49
Broadleaf evergreen     33
Other                   26
Bulb                    16
Vine                    13
Fern                    10
Name: Plant_Type, dtype: int64

#### Engineering Time - Column: "Zones"

In [12]:
zones = list(plant_df["Zones"])

min_zones = []
for plant in zones:
    min_zones.append(int(float(plant.split(" ")[0])))


print(f"Unique zone values to convert to temperatures: {set(min_zones)}")

Unique zone values to convert to temperatures: {2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}


In [13]:
zone_to_temp = {2: -45.6, 3: -40.0, 4: -34.4, 5: -28.9, 6: -23.3, 7: -17.8, 8: -12.2, 9: -6.7, 10: -1.1, 11: 4.4, 12: 10}

min_temp = []
for min_zone in min_zones:
    min_temp.append(zone_to_temp[min_zone])

plant_df["Min_Temp_Degrees_C"] = min_temp
plant_df["Min_Temp_Degrees_C"].value_counts()

-1.1     58
-6.7     26
 4.4     23
-12.2    12
-34.4     9
-23.3     7
-40.0     4
-45.6     3
-28.9     3
 10.0     1
-17.8     1
Name: Min_Temp_Degrees_C, dtype: int64

#### Engineering Time - Column: "Heights" and "Spreads"

Extracting the min and max values for both columns 

In [14]:
# get min and max height and min and max spread. 
heights, spreads = list(plant_df["Heights"]), list(plant_df["Spreads"]), 

min_heights, max_heights = [], []
for height in heights:
    min_heights.append(float(height.split(" ")[0]))
    max_heights.append(float(height.split(" ")[2]))

min_spreads, max_spreads = [], []
for spread in spreads:
    min_spreads.append(float(spread.split(" ")[0]))
    max_spreads.append(float(spread.split(" ")[2]))


plant_df["Min_Height"] = min_heights
plant_df["Max_Height"] = max_heights
plant_df["Min_Spread"] = min_spreads
plant_df["Max_Spread"] = max_spreads

In [15]:
# Now limit the max_height and spreads to 20 (to deal with outlier for modelling)

max_heights_capped = []
for height in max_heights:
    if height > 20:
        max_heights_capped.append(20)
    else:
        max_heights_capped.append(height) 

max_spreads_capped = []
for spread in max_spreads:
    if spread > 20:
        max_spreads_capped.append(20)
    else:
        max_spreads_capped.append(spread) 

plant_df["Max_Height_Capped"] = max_heights_capped
plant_df["Max_Spread_Capped"] = max_spreads_capped

#### Engineering Time - Column: "Bloom_Times" 
Converting this to a simpler categorical column as I did in Step 1


In [16]:
bloom_times = list(plant_df["Bloom_Times"])

non_flowering_terms = ["Non-flowering"]
rarely_flowering_terms = ["Rarely flowers indoors", "Rarely flowers"]

flower_or_not = []
for descript in bloom_times:
    if descript in non_flowering_terms:
        flower_or_not.append("No")
    elif descript in rarely_flowering_terms:
        flower_or_not.append("Rarely")
    else:
        flower_or_not.append("Yes")

plant_df["Flowers_Or_Not"] = flower_or_not
plant_df["Flowers_Or_Not"].value_counts()

Yes       114
Rarely     20
No         13
Name: Flowers_Or_Not, dtype: int64

#### Engineering Time - Column: "Leafs" and "Fruits"

Making these into 2 binary columns:  (colorful or not for leaves) and (gives fruit or does not give fruit)

In [17]:
display(plant_df["Leafs"].value_counts())
display(plant_df["Fruits"].value_counts())

Evergreen              75
None                   31
Colorful, Evergreen    21
Colorful               16
Fragrant, Evergreen     2
Fragrant                1
Good Fall               1
Name: Leafs, dtype: int64

None             113
Showy             24
Showy, Edible     10
Name: Fruits, dtype: int64

In [18]:
leafs = list(plant_df["Leafs"])
colorful_leaves = []
for descript in leafs:
    if "colorful" in descript.lower():
        colorful_leaves.append("Colorful")
    else: 
        colorful_leaves.append("Not Colorful")
plant_df["Colorful_Leaves"] = colorful_leaves


fruits = list(plant_df["Fruits"])
fruit_or_not = []
for descript in fruits:
    if descript == "None":
        fruit_or_not.append("No Fruit")
    else:
        fruit_or_not.append("Yes")
plant_df["Fruit_Or_Not"] = fruit_or_not

# check correct. 
plant_df[["Colorful_Leaves", "Fruit_Or_Not"]].value_counts() 

Colorful_Leaves  Fruit_Or_Not
Not Colorful     No Fruit        79
Colorful         No Fruit        34
Not Colorful     Yes             31
Colorful         Yes              3
dtype: int64

#### Perform Ordinal and One Hot Encoding 

In [19]:
# Ordinal encoding
sunlight_dict = {
    "Part shade" : 1, "Part shade to full shade" : 2, 
    "Full sun to part shade" : 3, "Full sun" : 4
}
watering_dict = {
    "Dry" : 1, "Dry to medium" : 2,
    "Medium" : 3, "Medium to wet" : 4, "Wet" : 5
}
maintenance_dict = {"Low" : 1, "Medium" : 2, "High" : 3}
flowers_or_not_dict = {"No" : 1, "Rarely" : 2, "Yes" : 3,}

plant_df["Sunlight_Ordinal"] = plant_df["Sunlight"].map(sunlight_dict)
plant_df["Watering_Ordinal"] = plant_df["Watering"].map(watering_dict)
plant_df["Maintenance_Ordinal"] = plant_df["Maintenance"].map(maintenance_dict)
plant_df["Flowers_Ordinal"] = plant_df["Flowers_Or_Not"].map(flowers_or_not_dict)

plant_df.head(2)

,Plant_Name,Plant_Type,Zones,Heights,Spreads,Bloom_Times,Sunlight,Watering,Maintenance,Leafs,...,Max_Spread,Max_Height_Capped,Max_Spread_Capped,Flowers_Or_Not,Colorful_Leaves,Fruit_Or_Not,Sunlight_Ordinal,Watering_Ordinal,Maintenance_Ordinal,Flowers_Ordinal
0,Aechmea,Other,10 to 11,1.00 to 3.00 feet,1.00 to 2.00 feet,Seasonal bloomer,Part shade,Medium,Medium,Evergreen,...,2.0,3.0,2.0,Yes,Not Colorful,No Fruit,1,3,2,3
1,Ardisia crenata,Broadleaf evergreen,8 to 10,4.00 to 5.00 feet,4.00 to 5.00 feet,May to June,Part shade to full shade,Medium,Medium,Evergreen,...,5.0,5.0,5.0,Yes,Not Colorful,Yes,2,3,2,3


In [20]:
# one hot encoding. 
df_onehot_encoded = pd.get_dummies(plant_df[["Plant_Type", "Colorful_Leaves", "Fruit_Or_Not"]], prefix=["Type", "Color", "Fruit"], drop_first=True)

plant_df_complete = plant_df.join(df_onehot_encoded).copy(deep=True)
plant_df_complete.head(2)

,Plant_Name,Plant_Type,Zones,Heights,Spreads,Bloom_Times,Sunlight,Watering,Maintenance,Leafs,...,Watering_Ordinal,Maintenance_Ordinal,Flowers_Ordinal,Type_Bulb,Type_Fern,Type_Herbaceous perennial,Type_Other,Type_Vine,Color_Not Colorful,Fruit_Yes
0,Aechmea,Other,10 to 11,1.00 to 3.00 feet,1.00 to 2.00 feet,Seasonal bloomer,Part shade,Medium,Medium,Evergreen,...,3,2,3,0,0,0,1,0,1,0
1,Ardisia crenata,Broadleaf evergreen,8 to 10,4.00 to 5.00 feet,4.00 to 5.00 feet,May to June,Part shade to full shade,Medium,Medium,Evergreen,...,3,2,3,0,0,0,0,0,1,1


#### Store the new df in a dataframe
- At this point I have generated all the new features I want. Of course, I will need to convert the categorical columns to numerical with ordinal or one-hot encoding, but this will be done in Steps 3 and 4 as required.  
- Now I will save this dataframe to the database (removing those columns not needed in Step 3 or Step 4). 

In [21]:
# renaming to remove spaces otherwise bad SQL practice.
plant_df_complete = plant_df_complete.rename(columns={
    "Type_Herbaceous perennial": "Type_Herbaceous_perennial", 
    "Color_Not Colorful": "Color_Not_Colorful"})

plant_df_complete.columns

Index(['Plant_Name', 'Plant_Type', 'Zones', 'Heights', 'Spreads',
       'Bloom_Times', 'Sunlight', 'Watering', 'Maintenance', 'Leafs', 'Fruits',
       'Min_Temp_Degrees_C', 'Min_Height', 'Max_Height', 'Min_Spread',
       'Max_Spread', 'Max_Height_Capped', 'Max_Spread_Capped',
       'Flowers_Or_Not', 'Colorful_Leaves', 'Fruit_Or_Not', 'Sunlight_Ordinal',
       'Watering_Ordinal', 'Maintenance_Ordinal', 'Flowers_Ordinal',
       'Type_Bulb', 'Type_Fern', 'Type_Herbaceous_perennial', 'Type_Other',
       'Type_Vine', 'Color_Not_Colorful', 'Fruit_Yes'],
      dtype='object')

In [22]:
columns_to_keep = [
    "Plant_Name",
    "Min_Temp_Degrees_C", "Min_Height", "Max_Height", "Min_Spread", "Max_Spread",
    "Max_Height_Capped", "Max_Spread_Capped", 
    "Sunlight_Ordinal", "Watering_Ordinal", "Maintenance_Ordinal",
    "Flowers_Ordinal", "Type_Bulb", "Type_Fern",
    "Type_Herbaceous_perennial", "Type_Other", "Type_Vine",
    "Color_Not_Colorful", "Fruit_Yes"
]

features_df = plant_df_complete[columns_to_keep].copy(deep=True) 
features_df.head()

,Plant_Name,Min_Temp_Degrees_C,Min_Height,Max_Height,Min_Spread,Max_Spread,Max_Height_Capped,Max_Spread_Capped,Sunlight_Ordinal,Watering_Ordinal,Maintenance_Ordinal,Flowers_Ordinal,Type_Bulb,Type_Fern,Type_Herbaceous_perennial,Type_Other,Type_Vine,Color_Not_Colorful,Fruit_Yes
0,Aechmea,-1.1,1.0,3.0,1.0,2.0,3.0,2.0,1,3,2,3,0,0,0,1,0,1,0
1,Ardisia crenata,-12.2,4.0,5.0,4.0,5.0,5.0,5.0,2,3,2,3,0,0,0,0,0,1,1
2,Euphorbia milii,-6.7,3.0,6.0,1.5,3.0,6.0,3.0,4,2,2,3,0,0,0,0,0,1,0
3,Ficus elastica,-1.1,50.0,100.0,50.0,100.0,20.0,20.0,1,3,1,2,0,0,0,0,0,1,0
4,Woodsia obtusa,-34.4,1.0,1.5,2.0,2.5,1.5,2.5,2,3,2,1,0,1,0,0,0,1,0


In [23]:
features_df.describe()
# looks good. 

,Min_Temp_Degrees_C,Min_Height,Max_Height,Min_Spread,Max_Spread,Max_Height_Capped,Max_Spread_Capped,Sunlight_Ordinal,Watering_Ordinal,Maintenance_Ordinal,Flowers_Ordinal,Type_Bulb,Type_Fern,Type_Herbaceous_perennial,Type_Other,Type_Vine,Color_Not_Colorful,Fruit_Yes
count,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000
mean,-7.804082,7.496599,13.867347,4.474490,8.353741,7.370748,5.496599,2.231293,2.952381,1.537415,2.687075,0.108844,0.068027,0.333333,0.176871,0.088435,0.748299,0.231293
std,12.961180,14.825382,26.840589,9.372602,16.616078,7.185552,5.838508,1.147171,0.514626,0.643962,0.628138,0.312508,0.252653,0.473016,0.382864,0.284898,0.435474,0.423101
min,-45.600000,0.000000,0.250000,0.250000,0.500000,0.250000,0.500000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-12.200000,1.000000,1.500000,1.000000,1.750000,1.500000,1.750000,1.000000,3.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000
50%,-1.100000,2.000000,4.000000,1.500000,3.000000,4.000000,3.000000,2.000000,3.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,-1.100000,6.000000,12.000000,3.000000,6.500000,12.000000,6.500000,3.000000,3.000000,2.000000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
max,10.000000,100.000000,200.000000,60.000000,100.000000,20.000000,20.000000,4.000000,5.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
c.execute("""DROP TABLE IF EXISTS plant_features""")

In [25]:
features_df.columns

Index(['Plant_Name', 'Min_Temp_Degrees_C', 'Min_Height', 'Max_Height',
       'Min_Spread', 'Max_Spread', 'Max_Height_Capped', 'Max_Spread_Capped',
       'Sunlight_Ordinal', 'Watering_Ordinal', 'Maintenance_Ordinal',
       'Flowers_Ordinal', 'Type_Bulb', 'Type_Fern',
       'Type_Herbaceous_perennial', 'Type_Other', 'Type_Vine',
       'Color_Not_Colorful', 'Fruit_Yes'],
      dtype='object')

In [26]:
c.execute("""
CREATE TABLE IF NOT EXISTS plant_features(
    Plant_Name TEXT PRIMARY KEY,
    Min_Temp_Degrees_C REAL,
    Min_Height REAL,
    Max_Height REAL,
    Min_Spread REAL,
    Max_Spread REAL,
    Max_Height_Capped REAL,
    Max_Spread_Capped REAL,
    Sunlight_Ordinal INTEGER,
    Watering_Ordinal INTEGER,
    Maintenance_Ordinal INTEGER,
    Flowers_Ordinal INTEGER,
    Type_Bulb INTEGER,
    Type_Fern INTEGER,
    Type_Herbaceous_perennial INTEGER,
    Type_Other INTEGER,
    Type_Vine INTEGER,
    Color_Not_Colorful INTEGER,
    Fruit_Yes INTEGER
    )
""")
conn.commit()
c.execute("""PRAGMA table_info("plant_features")""")
c.fetchall()

[(0, 'Plant_Name', 'TEXT', 0, None, 1),
 (1, 'Min_Temp_Degrees_C', 'REAL', 0, None, 0),
 (2, 'Min_Height', 'REAL', 0, None, 0),
 (3, 'Max_Height', 'REAL', 0, None, 0),
 (4, 'Min_Spread', 'REAL', 0, None, 0),
 (5, 'Max_Spread', 'REAL', 0, None, 0),
 (6, 'Max_Height_Capped', 'REAL', 0, None, 0),
 (7, 'Max_Spread_Capped', 'REAL', 0, None, 0),
 (8, 'Sunlight_Ordinal', 'INTEGER', 0, None, 0),
 (9, 'Watering_Ordinal', 'INTEGER', 0, None, 0),
 (10, 'Maintenance_Ordinal', 'INTEGER', 0, None, 0),
 (11, 'Flowers_Ordinal', 'INTEGER', 0, None, 0),
 (12, 'Type_Bulb', 'INTEGER', 0, None, 0),
 (13, 'Type_Fern', 'INTEGER', 0, None, 0),
 (14, 'Type_Herbaceous_perennial', 'INTEGER', 0, None, 0),
 (15, 'Type_Other', 'INTEGER', 0, None, 0),
 (16, 'Type_Vine', 'INTEGER', 0, None, 0),
 (17, 'Color_Not_Colorful', 'INTEGER', 0, None, 0),
 (18, 'Fruit_Yes', 'INTEGER', 0, None, 0)]

In [27]:
features_df.to_sql("plant_features", con=conn, if_exists="append", index=False)
test_saved_df = pd.read_sql_query("SELECT * FROM plant_features", conn)
test_saved_df.head() 

,Plant_Name,Min_Temp_Degrees_C,Min_Height,Max_Height,Min_Spread,Max_Spread,Max_Height_Capped,Max_Spread_Capped,Sunlight_Ordinal,Watering_Ordinal,Maintenance_Ordinal,Flowers_Ordinal,Type_Bulb,Type_Fern,Type_Herbaceous_perennial,Type_Other,Type_Vine,Color_Not_Colorful,Fruit_Yes
0,Aechmea,-1.1,1.0,3.0,1.0,2.0,3.0,2.0,1,3,2,3,0,0,0,1,0,1,0
1,Ardisia crenata,-12.2,4.0,5.0,4.0,5.0,5.0,5.0,2,3,2,3,0,0,0,0,0,1,1
2,Euphorbia milii,-6.7,3.0,6.0,1.5,3.0,6.0,3.0,4,2,2,3,0,0,0,0,0,1,0
3,Ficus elastica,-1.1,50.0,100.0,50.0,100.0,20.0,20.0,1,3,1,2,0,0,0,0,0,1,0
4,Woodsia obtusa,-34.4,1.0,1.5,2.0,2.5,1.5,2.5,2,3,2,1,0,1,0,0,0,1,0


In [28]:
c.close()